<a href="https://colab.research.google.com/github/moussaka217/repo/blob/main/fineTunningPhi2Ouss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate einops tqdm scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework

In [2]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 3.6 MB/s eta 0:00:00


In [3]:
import os
from dataclasses import dataclass, field
from typing import Optional

In [4]:
import torch
from datasets import load_dataset, load_from_disk
from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
)

In [5]:
import pandas as pd
dataSetQa = pd.read_csv('data/data.csv', delimiter=';', header=None)

In [6]:
dataSetQa.head(2)

,0,1
0,Comment mc2i intègre-t-elle le digital pour am...,mc2i utilise le digital pour optimiser les pr...
1,Quelle est l'approche de mc2i en matière de re...,mc2i s'engage activement dans des initiatives...


In [7]:
dataSetQa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       30 non-null     object
 1   1       30 non-null     object
dtypes: object(2)
memory usage: 608.0+ bytes


In [8]:
def format_row(row):
    question = row.iloc[0]
    answer = row.iloc[1]
    concat = f"[INST] {question} [/INST] {answer}"
    return concat

In [9]:
dataSetQa["Text"] = dataSetQa.apply(format_row, axis=1)

In [10]:
dataSetQa["Text"]

0     [INST] Comment mc2i intègre-t-elle le digital ...
1     [INST] Quelle est l'approche de mc2i en matièr...
2     [INST] Comment mc2i assure-t-elle l'équilibre ...
3     [INST] En quoi consiste l'initiative mc2i Init...
4     [INST] Quelle est la stratégie de mc2i pour fa...
5     [INST] Comment mc2i s'engage-t-elle en faveur ...
6     [INST] Quels sont les principaux domaines d'im...
7     [INST] Comment mc2i promeut-elle le bien-être ...
8     [INST] Quelle est la contribution de mc2i à l'...
9     [INST] Comment mc2i soutient-elle l'innovation...
10    [INST] Comment mc2i accompagne-t-elle ses clie...
11    [INST] Quelles mesures mc2i prend-elle pour as...
12    [INST] Comment mc2i favorise-t-elle l'innovati...
13    [INST] Quelle importance mc2i accorde-t-elle a...
14    [INST] Comment mc2i intègre-t-elle les princip...
15    [INST] Quelles actions mc2i entreprend-elle po...
16    [INST] Comment mc2i gère-t-elle les défis du t...
17    [INST] Quel est le rôle de l'intelligence 

In [11]:
new_df = dataSetQa[['Text']]

In [12]:
new_df.head(3)

,Text
0,[INST] Comment mc2i intègre-t-elle le digital ...
1,[INST] Quelle est l'approche de mc2i en matièr...
2,[INST] Comment mc2i assure-t-elle l'équilibre ...


In [13]:
new_df.to_csv("data/formatted_data.csv",index=False)

In [14]:
from tqdm.notebook import tqdm
from trl import SFTTrainer
from huggingface_hub import interpreter_login

In [15]:
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: ··········
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [16]:
training_dataset = load_dataset("csv",data_files = "data/formatted_data.csv",split = "train")

Generating train split: 0 examples [00:00, ? examples/s]

In [17]:
training_dataset

Dataset({
    features: ['Text'],
    num_rows: 30
})

In [20]:
base_model ="microsoft/phi-2"
new_model = "phi2-finetuned-tryout"
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = False
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config = bnb_config,
    trust_remote_code = True,
    flash_attn = True,
    flash_rotary = True,
    fused_dense = True,
    low_cpu_mem_usage = True,
    device_map = "auto",
    revision = "refs/pr/23"
    )

model.config.use_cache = False
model.config.pretraining_tp = 1

model = prepare_model_for_kbit_training(model , use_gradient_checkpointing=True)

training_arguments = TrainingArguments(
    output_dir = "./mhGPT",
    num_train_epochs = 2,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 32,
    evaluation_strategy = "steps",
    eval_steps = 1500,
    logging_steps = 15,
    optim = "paged_adamw_8bit",
    learning_rate = 2e-4,
    lr_scheduler_type = "cosine",
    save_steps = 1500,
    warmup_ratio = 0.05,
    weight_decay = 0.01,
    max_steps = -1
)

peft_config = LoraConfig(
    r = 32,
    lora_alpha = 64,
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM",
    target_modules = ["Wqkv", "fc1", "fc2"]
)

trainer = SFTTrainer(
    model = model,
    train_dataset = training_dataset,
    peft_config = peft_config,
    dataset_text_field = "Text",
    max_seq_length = 690,
    tokenizer = tokenizer,
    args = training_arguments
)

ValueError: 
                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit
                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping
                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom
                        `device_map` to `from_pretrained`. Check
                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                        for more details.
                        

In [ ]:
trainer.train()